Постановка задачи: многозадачное иерархическое обучение, то есть навешивание функций потерь на разные уровни сети (т. е. выходы промежуточных слоев). В качестве сети можно использовать предобученный трансформер. Идея иерархического обучения исходит из того, что процесс анализа текста и разбивается на уровни: морфологический, синтаксический, семантический. С помощью иерархии задач мы подталкиваем сеть к тому, чтобы разные уровни решали разные по иерархии задачи.

Что еще можно попробовать:

1. Иногда в нижних уровнях может использоваться информация из более высоких уровней, поэтому явное разбиение сети на уровни может оказаться слишком ограничивающим и привести к тому, что нижний уровень будет пытаться решать задачи более высокого уровня. Поэтому можно либо делать конкатенацию выходов всех слоев, либо применять ко выходам всех слоев (на одном и том же токене) self-attention. При этом можно использовать не сами выходы слоев, а разность между токеном на текущем слое и предыдущем (т. к. сами слои работают аддитивно).

2. Можно попробовать суммировать на входе все токены, относящиеся к одному и тому же слову - либо перед первым слоем, либо перед каким-то из промежуточных слоев, где заканчивается анализ частей слов и начинается анализ слов целиком. Так мы ускорим работу сети, поскольку скорость работы квадратично зависит от числа токенов.

3. Аддитивная работа каждого слоя трансформера (output = f(input) + input) может быть неоптимальна. Попробуйте суммировать 10 картинок, и получите нечто непонятное. Можно изменить структуру трансформера: каждый слой теперь будет выдавать значения, но не будет прибавлять их ко входу. При этом в начало каждого слоя добавляется еще один блок, который работает с помощью attention, но не по оси "времени", а по оси номера слоя. Тем самым каждый слой сам выбирает, результаты работы каких слоев он должен использовать.

Плюс есть еще пара идей, которые будут далее.

--------

Обзор литературы:

[TinyBERT](https://arxiv.org/abs/1909.10351)


Двухэтапная дистилляция: сначала из модели, обученной на MLM, затем из модели, дообученной для специфичной задачи. На обоих шагах используется один и тот же студент, то есть он обучается в два этапа.

[ERNIE 2.0](https://arxiv.org/abs/1907.12412)


Модель каким-то образом (нужно читать подробнее) сама себе создает задачи в self-supervised режиме. Таким образом, модель обучается сначала на одгной задаче, затем на двух, и так далее.

[ERNIE 3.0](https://arxiv.org/abs/2107.02137)


Обучение модели с многими головами, где каждая голова также является трансформером. Одновременное обучение этой сети как в режиме GPT, так и в режиме BERT. Специальное маскирование в обучающих данных с помощью графов знаний.

[MKD](https://arxiv.org/abs/1911.03588)


Дистилляция: один учитель, обученный на multi-task, один ученик с многими головами для разных задач.

[Meta-KD](https://arxiv.org/abs/2012.01266)


Дистилляция: один учитель, обученный на multi-task, модель-ученик для какой-то специфической задачи.

[MergeDistill](https://arxiv.org/abs/2106.02834)

Дистилляция из многих моделей-учителей (одноязычных и мультиязычных) в одну модель-студент (многоязычную).

--------

Другие статьи, которые обсуждались:

- [Assessing Phrasal Representation and Composition in Transformers](https://arxiv.org/abs/2010.03763)
- [Language Models are not Models of Language](https://arxiv.org/abs/2112.07055)
- [HRKD: Hierarchical Relational Knowledge Distillation](https://arxiv.org/abs/2110.08551)
- [How not to Lie with a Benchmark: Rearranging NLP Leaderboards](https://arxiv.org/abs/2112.01342)
- [Relational inductive biases, deep learning, and graph networks](https://arxiv.org/abs/1806.01261)
- [Implicit MLE: Backpropagating Through Discrete Exponential Family Distributions](https://arxiv.org/abs/2106.01798)
- [Every Model Learned by Gradient Descent Is Approximately a Kernel Machine](https://arxiv.org/abs/2012.00152)
- [Gradient Descent Finds Global Minima of Deep Neural Networks](https://arxiv.org/abs/1811.03804)

## Иерархический трансформер

По-моему трансформеры не соответствуют двум важным свойствам языка:

1. Иерархичность. Трансформер - это графовая сеть, в которой все токены связаны со всеми. В языке, напротив, связность иерархическая. Из-за этого трансформер может работать во-первых медленнее, во-вторых менее эффективно, так как его структура не отражает структуру языка. Надо подумать над возможностью внести какие-то (желательно небольшие) изменения в трансформер, чтобы превратить его в иерархический. В [этой статье](https://arxiv.org/pdf/2110.13711.pdf) трансформер строится иерархически, наподобие U-Net. Только это сделано по-простому: несколько токенов объединяются в один. ля языка это может быть не лучший подход, так как в тексте иерархия устроена по-друогму: например, два предложения имеют разную длину, но являются элементами одного уровня иерархии.

2. Расстояние между словами не всегда важно само по себе. Идея использовать позиционные эмбеддинги в трансформерах кажется искусственной и не соответствующей свойствам языка. Часто бывает так, что смысл текста не изменится, если переставить некоторые предложения местами, но позиционные эмбеддинги при этом изменятся полностью. Как вариант, можно попробовать в качестве входных и выходных данных трансформер-блока использовать не только эмбеддинги токенов, но и эмбеддинги пар токенов, описывающие связь между ними. В этом случае позиционные эмбеддинги вообще могут не понадобиться, а трансформер станет полноценной графовой сетью.

Чем ближе структура модели к структуре моделируемых данных, тем модель более эффективна и интерпретируема, и трансформер в этом плане далек от оптимальной модели.

Можно ли как-то закодировать иерархию в тексте без явного построения синтаксического дерева? Возможно, что да. Каждому токену $T$ можно сопоставить не только вектор $V_T$, кодирующий его значение (как в обычном трансформере), но плюс к этому еще один вектор $H_T$, кодирующий его положение в иерархии. Можно считать, что токены ;$T_1$ и $T_2$ находятся в иерархии тем ближе друг к другу, чем ближе вектора $H_{T_1}$ и $H_{T_2}$. Тем самым мы получим soft-иерархию (нечеткую), которая может хранить много информации и дифференцируема.

Далее, можно наложить на токены $V_T$ регуляризацию, которая требует, чтобы вектора $V_{T_1}$ и $V_{T_2}$ были тем ближе друг к другу, чем ближе вектора $H_{T_1}$ и $H_{T_2}$. Представим, что на высоком уровне (в последних слоях трансформера) мы хотим работать со словосочетаниями или даже предложениями как с сущностями. Тогда мы хотим рассчитывать attention не между отдельными токенами, а между словосочетаниями или предложениями. Предложенная выше регуляризация обеспечит тот факт, что эмбеддинги всех токенов в предложении будут похожи, и "внимание", рассчитанное между любой парой токенов из двух предложений $A$ и $B$ будут похожи.

Если же нам требуется решать token-level задачи, то можно после высокоуровневого анализа затем снова вернуться к уровню токенов, как в U-Net.

Все предложенное выше имеет своей целью позволить трансформеру работать как иерархическая графовая сеть, не выстраивая иерархию в явном виде (например, через argmax), чтобы упростить задачу обратного распространения ошибки.

После обучения такой моделью можно делать легковесный инференс лишь с небольшой потерей точности, если объединять похожие токены, для которых близки значения $H_{T_1}$ и $H_{T_2}$, методом усреднения (как в агломеративной иерархической кластеризации).